In [8]:
import torchvision.models as models
import torch
import torch.nn.functional as F
from torchvision import transforms
from PIL import Image
import copy
import numpy as np
from torch import nn
from dataset import IDRiD_Dataset
from dataset_techers import IDRiD_Dataset_teacher
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from pipe_models import MTL
device

device(type='cpu')

In [9]:
rx=450
ry=300
old_x=4288
old_y=2848
Rx=rx/old_x
Ry=ry/old_y

data_transformer = transforms.Compose([transforms.Resize((rx,ry)),transforms.ToTensor(),
                                       transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
train_ds = IDRiD_Dataset_teacher(data_transformer,'train')
train_dl = DataLoader(train_ds,batch_size=32,shuffle=True)
mtl = MTL('M2')


mtl=mtl.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(filter(lambda p: p.requires_grad, mtl.parameters()),
                                weight_decay=1e-6,
                                momentum=0.9,
                                lr=1e-3,
                                nesterov=True)
scheduler = ReduceLROnPlateau(optimizer,
                                  factor=0.5,
                                  patience=3,
                                  min_lr=1e-7,
                                  verbose=True)
tasks=[[0,1,2]]

In [10]:
mtl.fit(train_dl,optimizer,scheduler,criterion,tasks,1,Rx,Ry)

Epoch: 1 
|████████████████████████████████████████████████████████████████████████████████████████████████████| 100.00%
Total Loss: 22269.6431933156
Loss0: 20.5083072791469 
Loss1: 19.158762700142013  
Loss2: 22229.97612333631
Loss3: 1.9247028116737628
Saved best model weights!


In [7]:
import pandas as pd
mtl=MTL()
mtl.load_state_dict(torch.load("./weights"+str(tasks)+".pt"))
mtl.eval()
data=pd.DataFrame()
z = []
for i, (imgs, retinopathy_label, macular_edema_label, fovea_center_labels, optical_disk_labels) in enumerate(train_ds):
    one_row = mtl.forward(imgs[None,:])
    z.append(torch.concat(one_row).detach().numpy())
    if i==3:
        break
data = pd.DataFrame(z)
data.to_csv('./IDRID/Labels/teacher_train_predictions'+str(tasks)+'.csv')